In [53]:
'''Downlod Videos'''


# from pytube import YouTube
# yt = YouTube('https://youtu.be/jdWZjGLxcaY')
# print(yt.title)
# print(yt.thumbnail_url)
# stream = yt.streams.filter(progressive=True, file_extension='mp4').first()
# stream.download()

'Downlod Videos'

In [51]:
# '''Creating Mapping.csv file'''


# from os import listdir
# from os.path import isfile, join

# mypath1 = '/Users/mac16_anyang/Documents/Data Science/video_processing'
# onlyfiles = [h for h in listdir(mypath1) if isfile(join(mypath1, h))]

# from os import walk

# h = []
# for (dirpath, dirnames, filenames) in walk(mypath1):
#     h.extend(filenames)
#     break
    

# mypath2 = '/Users/mac16_anyang/Documents/Data Science/video_processing/tom'
# onlyfiles = [t for t in listdir(mypath2) if isfile(join(mypath2, t))]

# t = []
# for (dirpath, dirnames, filenames) in walk(mypath2):
#     t.extend(filenames)
#     break    
    
    
# import pandas as pd
# df = pd.DataFrame(h)
# df['Class'] = 0
# df.rename(columns={0:'Image_ID'},inplace = True)


# df1 = pd.DataFrame(t)
# df1['Class'] = 1
# df1.rename(columns={0:'Image_ID'},inplace = True)


# data = pd.concat([df,df1],axis=0)
# data = data.set_index('Image_ID')
# data.to_csv('mapping.csv')


In [18]:
# pip install opencv-python

import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils     #  utility package for keras
from skimage.transform import resize   # for resizing images


Step – 1: Read the video, extract frames from it and save them as images

In [55]:
'''Capture Images from Video'''


# videoFile = "tom1.mp4"
# cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
# frameRate = 50 #frame rate cap.get(5)
# x = 660
# while(cap.isOpened()):
#     frameId = cap.get(1) #current frame number
#     ret, frame = cap.read()
#     if (ret != True):
#         break
#     if (frameId % math.floor(frameRate) == 0):
#         filename ="frame%d.jpg" % x;x+=1
#         cv2.imwrite(filename, frame)
# cap.release()
# print ("Done!")

# img = plt.imread('frame0.jpg')   # reading image using its name
# plt.imshow(img)

'Capture Images from Video'

Step – 2: Label a few images for training the model

In [2]:
# data = pd.read_csv('mapping.csv')     # reading the csv file
# data = data[data.Image_ID != '.DS_Store']
# data.head()      # printing first five rows of the file

# X = [ ]     # creating an empty array
# for img_name in data.Image_ID:
#     img = plt.imread(img_name)
#     X.append(img)  # storing each image in array X
# X = np.array(X)    # converting list to array


# y = data.Class
# dummy_y = np_utils.to_categorical(y)    # one hot encoding Classes


# image = []
# for i in range(0,X.shape[0]):
#     a = resize(X[i], preserve_range=True, output_shape=(224,224)).astype(int)      # reshaping to 224*224*3
#     image.append(a)
# X = np.array(image)


# from keras.applications.vgg16 import preprocess_input
# X = preprocess_input(X)      # preprocessing the input data


# from sklearn.model_selection import train_test_split
# X_train, X_valid, y_train, y_valid = train_test_split(X, dummy_y, test_size=0.3, random_state=42)    # preparing the validation set




Step 3: Building the model


In [6]:
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout

# i.Make predictions using this model, and then use those features to retrain the model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))    # include_top=False to remove the top layer


X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)
X_train.shape, X_valid.shape


X_train = X_train.reshape(885, 7*7*512)      # converting to 1-D
X_valid = X_valid.reshape(380, 7*7*512)



train = X_train/X_train.max()      # centering the data
X_valid = X_valid/X_train.max()


# ii. Building the model
model = Sequential()
model.add(InputLayer((7*7*512,)))    # input layer
model.add(Dropout(0.2)) # dropout layer
model.add(Dense(units=1024, activation='sigmoid')) # hidden layer
model.add(Dropout(0.2)) # dropout layer
model.add(Dense(2, activation='softmax'))    # output layer

model.summary()


# iii. Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# iv. Training the model
model.fit(train, y_train, epochs=30, validation_data=(X_valid, y_valid))





# predictions = model.predict_classes(test_image)

# scores = model.evaluate(test_image, test_y)
# print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))




Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout (Dropout)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              25691136  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2050      
Total params: 25,693,186
Trainable params: 25,693,186
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
28/28 [==============================] - 2s 76ms/step - loss: 0.1737 - accuracy: 0.9209 - val_loss: 0.0196 - val_accuracy: 1.0000
Epoch 2/30
28/28 [==============================] - 2s 74ms/step - loss: 0.0147 - accuracy: 0.9966 - 

In [132]:
# his = pd.DataFrame(model.history.history)
# his.plot.line()

In [76]:
mypath1 = '/Users/mac16_anyang/Documents/Data Science/video_processing'
# onlyfiles = [h for h in listdir(mypath1) if isfile(join(mypath1, h))]

from os import walk

h = []
file = []
for (dirpath, dirnames, filenames) in walk(mypath1):
    h.extend(filenames)
    break
    
for name in h:
        if name.split('.')[-1] == 'jpg':
            if name.split('.')[0] == 'cat':
                file.append([name,0])
            elif name.split('.')[0] == 'dog':
                file.append([name,1])

df = pd.DataFrame(file)
df.columns = ['name','label']

X = [ ]     # creating an empty array
for img_name in df.name:
    img = plt.imread(img_name)
    X.append(img)  # storing each image in array X
X = np.array(X)    # converting list to array


In [93]:

y = df.label
dummy_y = np_utils.to_categorical(y) 

image = []
for i in range(0,X.shape[0]):
    a = resize(X[i], preserve_range=True, output_shape=(224,224)).astype(int)      # reshaping to 224*224*3
    image.append(a)
X = np.array(image)


from keras.applications.vgg16 import preprocess_input
X = preprocess_input(X)      # preprocessing the input data


from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, dummy_y, test_size=0.3, random_state=42)   

In [96]:
# from keras.models import Sequential
# from keras.applications.vgg16 import VGG16
# from keras.layers import Dense, InputLayer, Dropout

# # i.Make predictions using this model, and then use those features to retrain the model
# base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))    # include_top=False to remove the top layer


# X_train = base_model.predict(X_train)
# X_valid = base_model.predict(X_valid)
# X_train.shape, X_valid.shape


# X_train = X_train.reshape(len(X_train), 7*7*512)      # converting to 1-D
# X_valid = X_valid.reshape(len(X_valid), 7*7*512)



# train = X_train/X_train.max()      # centering the data
# X_valid = X_valid/X_train.max()


# ii. Building the model
model = Sequential()
model.add(InputLayer((7*7*512,)))    # input layer
model.add(Dropout(0.2)) # dropout layer
model.add(Dense(units=1024, activation='sigmoid')) # hidden layer
model.add(Dropout(0.2)) # dropout layer
model.add(Dense(2, activation='softmax'))    # output layer

model.summary()


# iii. Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# iv. Training the model
model.fit(train, y_train, epochs=30, validation_data=(X_valid, y_valid))


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout (Dropout)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              25691136  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2050      
Total params: 25,693,186
Trainable params: 25,693,186
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
175/175 [==============================] - 13s 75ms/step - loss: 0.1255 - accuracy: 0.9495 - val_loss: 0.0736 - val_accuracy: 0.9696
Epoch 2/30
175/175 [==============================] - 13s 75ms/step - loss: 0.0353 - accuracy: 0.9

In [123]:
mypath1 = '/Users/mac16_anyang/Documents/Data Science/video_processing'
# onlyfiles = [h for h in listdir(mypath1) if isfile(join(mypath1, h))]

from os import walk

h2 = []
file2 = []
for (dirpath, dirnames, filenames) in walk(mypath1):
    h2.extend(filenames)
    break
    
for name in h2:
        if name.split('.')[-1] == 'jpg':
            if name.split('.')[0] == 'cat':
                file2.append([name,0])
            elif name.split('.')[0] == 'dog':
                file2.append([name,1])

df2 = pd.DataFrame(file2)
df2.columns = ['name','label']

X = [ ]     # creating an empty array
for img_name in df2.name:
    img = plt.imread(img_name)
    X.append(img)  # storing each image in array X
X = np.array(X)    # converting list to array


In [129]:
y = df2.label
dummy_y = np_utils.to_categorical(y) 

image = []
for i in range(0,X.shape[0]):
    a = resize(X[i], preserve_range=True, output_shape=(224,224)).astype(int)      # reshaping to 224*224*3
    image.append(a)
X = np.array(image)


from keras.applications.vgg16 import preprocess_input
X = preprocess_input(X)      # preprocessing the input data
X_test = base_model.predict(X)


X_test = X_test.reshape(len(X_test), 7*7*512)      # converting to 1-D
test = X_test/X_test.max()

In [130]:






predictions = model.predict_classes(test)

scores = model.evaluate(test, dummy_y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

3/3 [==============================] - 0s 6ms/step - loss: 0.3905 - accuracy: 0.9459
accuracy: 94.59%


array([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0])